In [1]:

from selenium import webdriver
import selenium
from io import StringIO
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = r"F:\sa\chromedriver.exe"


In [83]:
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
wd.get('https://google.com')

In [84]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Dhokla images')

In [85]:
wd.quit()

In [2]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [3]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [5]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=130):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [20]:
import io
import os
import time
import requests
#from io import image
from PIL import Image 
import hashlib
search_term='shrikhand'
search_and_download(
    search_term=search_term,
    driver_path=DRIVER_PATH
)

Found: 100 search results. Extracting links from 0:100
Found: 130 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTcx53XDwuOUAS8iGeQv2gsZBiXSrhdjTjrG8QrZ-2B6p3Au3Od&usqp=CAU - as ./images\shrikhand\83d83da467.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSSbvJ3ZBg2nDV_42Lz-R-lf0vkNjj8PitqUoPLOD9rNzCypX6T&usqp=CAU - as ./images\shrikhand\9a23866745.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR3S6r7hYpCzhV3f6sQCTlXCK6qosESPoSC9vy0XdZvVrwURBgh&usqp=CAU - as ./images\shrikhand\8924a57a7b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTwon4ZlNAjmr-BIWSjUCU9NkO_KfFyD4BsfG3UZrt7Af8ghue8&usqp=CAU - as ./images\shrikhand\2fa281c4be.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT2meQ5fxAE_GP7VNTDURzKJdl9H5RrIQUzg_zvpEWpSIP3chVd&usqp=CAU - as ./images\shrikhand\36b349302e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=

SUCCESS - saved https://mildlyindian.com/wp-content/uploads/2019/01/Fruit-Shrikhand.jpg - as ./images\shrikhand\9ade54473a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQq8QAksBcb9-qCVDeyg6TMLCRMlUygRcvztlnhfFRSpv75a-9w&usqp=CAU - as ./images\shrikhand\8d5096a775.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQuoRE5H2IeOY06d_Ia-AtRYaCgno4D18A0T17uKeNAUtpWgUhX&usqp=CAU - as ./images\shrikhand\13112f5fe8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSAB2FYnPLxbE5ns7VZKE09de3IBEocuWqRScvhv2c3yr2OD9iq&usqp=CAU - as ./images\shrikhand\cb1d9fd60f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSBxAc6TemoTRIDUgd2V3CEU40VwOTmPr2sJxRMrXj8xpxhoArv&usqp=CAU - as ./images\shrikhand\dc4c5bf854.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQf8VU05zORUTclM-vVwQiopKMDLu4V20a28V1lmi3GfW4lkvNN&usqp=CAU - as ./images\shrikhand\813022404f.jpg
SUCCESS - saved ht

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT8wuOCnBbHGuMGQPnkS58qO0LzKqQEKtCN2AsKhnEZrzR3G-s3&usqp=CAU - as ./images\shrikhand\c27b59d86f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTpQidqW85IyWRr23chbiJBX_fK4GupF1cxbSlBJNx4ibF2onKZ&usqp=CAU - as ./images\shrikhand\0263252462.jpg
SUCCESS - saved https://www.whiskaffair.com/wp-content/uploads/2014/05/Mango-Shrikhand-4.jpg - as ./images\shrikhand\7c8f8ebeac.jpg
SUCCESS - saved https://i2.wp.com/www.vegrecipesofindia.com/wp-content/uploads/2015/04/kesar-elaichi-shrikhand.jpg - as ./images\shrikhand\1a2959732e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQiCvI8SD2Eu_CiUTu5_aYbWIu6AXPCcr3akgdCq68QKKvtenFe&usqp=CAU - as ./images\shrikhand\e7972d9f0c.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/thumb/5/5c/Shrikhand_london_kastoori.jpg/1200px-Shrikhand_london_kastoori.jpg - as ./images\shrikhand\fc68a29e67.jpg
SUCCESS - saved https

In [21]:
d = ['sa', 'sad','asd']
dt ={}
for i, val in enumerate(d):
    dt[val]=i

In [22]:
dt

{'sa': 0, 'sad': 1, 'asd': 2}